# Pseudo
- Select points in RGB
- For each point, find k nearest neighbours assume evenly distributed
- Score based on difference between

# Focus
- Minimise class instantiation
- Numpy arrays

In [5]:
import numpy as np

In [6]:
N = 10

In [10]:
rgb_points = np.random.random(size=[N, 3])

In [12]:
rgb_points

array([[0.45221969, 0.88807475, 0.48498694],
       [0.07916363, 0.84307152, 0.60840341],
       [0.25219011, 0.17505316, 0.22348611],
       [0.96849772, 0.42076737, 0.68864667],
       [0.47782062, 0.24419521, 0.26092017],
       [0.81078311, 0.3070404 , 0.98801979],
       [0.44567367, 0.19211148, 0.9953147 ],
       [0.26849204, 0.94067574, 0.18614681],
       [0.44718872, 0.14234202, 0.25236196],
       [0.76068446, 0.08646759, 0.62321276]])

In [14]:
pip install colormath

Defaulting to user installation because normal site-packages is not writeable
  Using cached colormath-3.0.0-py3-none-any.whl
  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_diff import delta_e_cie1994
from colormath.color_conversions import convert_color

In [18]:
help(sRGBColor)

Help on class sRGBColor in module colormath.color_objects:

class sRGBColor(BaseRGBColor)
 |  sRGBColor(rgb_r, rgb_g, rgb_b, is_upscaled=False)
 |  
 |  Represents an sRGB color.
 |  
 |  .. note:: If you pass in upscaled values, we automatically scale them
 |      down to 0.0-1.0. If you need the old upscaled values, you can
 |      retrieve them with :py:meth:`get_upscaled_value_tuple`.
 |  
 |  :ivar float rgb_r: R coordinate
 |  :ivar float rgb_g: G coordinate
 |  :ivar float rgb_b: B coordinate
 |  :ivar bool is_upscaled: If True, RGB values are between 1-255. If False,
 |      0.0-1.0.
 |  
 |  Method resolution order:
 |      sRGBColor
 |      BaseRGBColor
 |      ColorBase
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  conversion_matrices = {'rgb_to_xyz': array([[0.412424 , 0.357579 , 0.1...
 |  
 |  native_illuminant = 'd65'
 |  
 |  rgb_gamma = 2.2
 |  
 |  ----------------------------------------------------------------------
 |  Methods in

In [22]:
%%timeit
sRGBColor(*rgb_points[0])

1.41 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [20]:
k = sRGBColor(*rgb_points[0])
k

sRGBColor(rgb_r=0.4522196878124888,rgb_g=0.8880747473857142,rgb_b=0.4849869429608783)

In [73]:
khx = k.get_rgb_hex()
khx

'#73e27c'

In [75]:
%%timeit
k.new_from_rgb_hex(khx)

1.53 µs ± 6.53 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%%timeit
convert_color(k, LabColor)

22.9 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
l1 = convert_color(k, LabColor)
l2 = convert_color(k, LabColor)

In [29]:
%%timeit
delta_e_cie1994(l1,l2)

63.8 µs ± 767 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [31]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.6.3-cp39-cp39-manylinux1_x86_64.whl (27.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
from scipy.optimize import minimize

In [39]:
help(minimize)

Help on function minimize in module scipy.optimize._minimize:

minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
    Minimization of scalar function of one or more variables.
    
    Parameters
    ----------
    fun : callable
        The objective function to be minimized.
    
            ``fun(x, *args) -> float``
    
        where ``x`` is an 1-D array with shape (n,) and ``args``
        is a tuple of the fixed parameters needed to completely
        specify the function.
    x0 : ndarray, shape (n,)
        Initial guess. Array of real elements of size (n,),
        where 'n' is the number of independent variables.
    args : tuple, optional
        Extra arguments passed to the objective function and its
        derivatives (`fun`, `jac` and `hess` functions).
    method : str or callable, optional
        Type of solver.  Should be one of
    
            - 'Nelder-Mead' :ref:`(see her

In [56]:
from scipy.spatial import cKDTree

In [57]:
rgb_points

array([[0.45221969, 0.88807475, 0.48498694],
       [0.07916363, 0.84307152, 0.60840341],
       [0.25219011, 0.17505316, 0.22348611],
       [0.96849772, 0.42076737, 0.68864667],
       [0.47782062, 0.24419521, 0.26092017],
       [0.81078311, 0.3070404 , 0.98801979],
       [0.44567367, 0.19211148, 0.9953147 ],
       [0.26849204, 0.94067574, 0.18614681],
       [0.44718872, 0.14234202, 0.25236196],
       [0.76068446, 0.08646759, 0.62321276]])

In [61]:
Tree = cKDTree(rgb_points)

In [62]:
# 2 nearest neighbours
# find the neighbours for five random points
dists, inds = Tree.query(np.random.random([5,3]), k=2)

In [63]:
inds

array([[6, 2],
       [0, 3],
       [0, 7],
       [0, 3],
       [7, 0]])

In [64]:
%%timeit
cKDTree(rgb_points)

15.1 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [65]:
%%timeit
Tree.query(np.random.random([5,3]), k=2)

23.3 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [83]:
def norm(arr):
    return np.sqrt(np.sum(np.square(arr)))

In [264]:
def score(lab_objs, neighbour_inds):
    
    # Calculate norm for all nearest differences
    sum_diff2 = 0
    for i, inds in enumerate(neighbour_inds):
        for j in inds[1:]: # ignore 0th nearest colour
            sum_diff2 += np.square(delta_e_cie1994(lab_objs[i], lab_objs[j]))
    
    score = -np.sqrt(sum_diff2)
    return score

def fun(C):
    # input array of colours
    # reshape to handle
    N = C.size//3
    C = C.reshape(N,3)
    
    # instantiate
    labs = [convert_color(sRGBColor(*rgb), LabColor) for rgb in C]
    
    # nearest neighbours
    rgb_tree = cKDTree(C)
    _, rgb_neighbour_inds = rgb_tree.query(C, k=N)
    
    obj = score(labs, rgb_neighbour_inds)
    
    # jacobian
    step = 0.01
    
    # Add 1 to each in RGB for each colour
    # dx00, dx01, dx02, dx10, ... d colour matrix
    labs_d = [[convert_color(sRGBColor(*np.add(rgb, np.roll([step, 0, 0], i))), LabColor) 
               for i in [0,1,2]]
                   for rgb in C] 
    
    def stitch(orig, newel, newind):
        return orig[:newind] + [newel] + orig[newind+1:]
    
    jac = [None for i in range(np.size(labs_d))]
    ji = 0
    for i, lab_di in enumerate(labs_d):
        for l in lab_di:
            obj_d = score(stitch(labs, l, i), rgb_neighbour_inds)
            jac[ji] = (obj_d - obj)/step
            
            ji += 1
        
    return obj, jac

In [265]:
%%timeit
_, p = fun(rgb_points)

191 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [266]:
import time

In [277]:
pts = np.random.random([10,3]).reshape(-1)

In [280]:
ti = time.time()
sol = minimize(fun, pts, jac=True, method='L-BFGS-B', bounds=[(0,1) for i in pts])
total = time.time() - ti
print(total)

7.087123870849609


In [281]:
sol

      fun: -888.4722683060419
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([   1.58632932, -104.93079655,    6.75404442,  336.47567196,
        651.2944585 ,  432.06520214,    4.22924178,   28.39301349,
        -43.33815443,  -23.49893606,  -91.7664435 ,    6.2756935 ,
        336.47567196,  651.2944585 ,  432.06520214,  336.50686789,
        651.4073017 ,  432.13031486,  -23.49893618,  -91.76655292,
          6.27569368,    1.58635976, -104.89493359,    6.75417196,
          1.58632932, -104.93079655,    6.75404442,    4.22924178,
         28.39301525,  -43.33816952])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 37
      nit: 7
     njev: 37
   status: 0
  success: True
        x: array([0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.])

In [269]:
help(minimize)

Help on function minimize in module scipy.optimize._minimize:

minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
    Minimization of scalar function of one or more variables.
    
    Parameters
    ----------
    fun : callable
        The objective function to be minimized.
    
            ``fun(x, *args) -> float``
    
        where ``x`` is an 1-D array with shape (n,) and ``args``
        is a tuple of the fixed parameters needed to completely
        specify the function.
    x0 : ndarray, shape (n,)
        Initial guess. Array of real elements of size (n,),
        where 'n' is the number of independent variables.
    args : tuple, optional
        Extra arguments passed to the objective function and its
        derivatives (`fun`, `jac` and `hess` functions).
    method : str or callable, optional
        Type of solver.  Should be one of
    
            - 'Nelder-Mead' :ref:`(see her

In [256]:
convert_color(sRGBColor(1,1,1), LabColor)

LabColor(lab_l=99.99998453333127,lab_a=-0.0004593894083471106,lab_b=-0.008561457924405325)

In [231]:
p

[3.805288282313768,
 25.25580234400877,
 2.0150847086000567,
 -0.7777250308095063,
 -0.46180729537184106,
 24.499089850903033,
 -79.58374107350608,
 48.13470090912517,
 -25.58658850518043,
 47.34051963212096,
 50.61165929707272,
 -45.927066028562535,
 13.971977085085996,
 -29.1613764711542,
 -33.754759675287005,
 1.8559304955459766,
 7.0390977262349,
 14.898147010927687,
 -21.730393594009456,
 -14.75110105736519,
 -2.023897871433178,
 3.072440574695179,
 39.79631617494306,
 -5.2801186066062655,
 4.53005382841809,
 -4.270033598939449,
 -9.121722662075626,
 2.5856275491491942,
 -15.324780147679462,
 33.347545937087375]

In [180]:
step = 0.1

In [181]:
labs_d = [np.add(rgb, np.roll([step, 0, 0], i)) for rgb in C for i in [0,1,2]]
labs_d

[array([0.55221969, 0.88807475, 0.48498694]),
 array([0.45221969, 0.98807475, 0.48498694]),
 array([0.45221969, 0.88807475, 0.58498694]),
 array([0.17916363, 0.84307152, 0.60840341]),
 array([0.07916363, 0.94307152, 0.60840341]),
 array([0.07916363, 0.84307152, 0.70840341]),
 array([0.35219011, 0.17505316, 0.22348611]),
 array([0.25219011, 0.27505316, 0.22348611]),
 array([0.25219011, 0.17505316, 0.32348611]),
 array([1.06849772, 0.42076737, 0.68864667]),
 array([0.96849772, 0.52076737, 0.68864667]),
 array([0.96849772, 0.42076737, 0.78864667]),
 array([0.57782062, 0.24419521, 0.26092017]),
 array([0.47782062, 0.34419521, 0.26092017]),
 array([0.47782062, 0.24419521, 0.36092017]),
 array([0.91078311, 0.3070404 , 0.98801979]),
 array([0.81078311, 0.4070404 , 0.98801979]),
 array([0.81078311, 0.3070404 , 1.08801979]),
 array([0.54567367, 0.19211148, 0.9953147 ]),
 array([0.44567367, 0.29211148, 0.9953147 ]),
 array([0.44567367, 0.19211148, 1.0953147 ]),
 array([0.36849204, 0.94067574, 0.

In [150]:
a = [1,2,3,4,5]
b = [6,7,8,9,0]

In [151]:
a[:2] + [b[2]] + a[3:]

[1, 2, 8, 4, 5]

In [147]:
[a for l in [0,1,2] for a in [3,4,5]]

[3, 4, 5, 3, 4, 5, 3, 4, 5]

In [145]:
rgb_points

array([[0.45221969, 0.88807475, 0.48498694],
       [0.07916363, 0.84307152, 0.60840341],
       [0.25219011, 0.17505316, 0.22348611],
       [0.96849772, 0.42076737, 0.68864667],
       [0.47782062, 0.24419521, 0.26092017],
       [0.81078311, 0.3070404 , 0.98801979],
       [0.44567367, 0.19211148, 0.9953147 ],
       [0.26849204, 0.94067574, 0.18614681],
       [0.44718872, 0.14234202, 0.25236196],
       [0.76068446, 0.08646759, 0.62321276]])

In [146]:
rgb_points.reshape(-1)

array([0.45221969, 0.88807475, 0.48498694, 0.07916363, 0.84307152,
       0.60840341, 0.25219011, 0.17505316, 0.22348611, 0.96849772,
       0.42076737, 0.68864667, 0.47782062, 0.24419521, 0.26092017,
       0.81078311, 0.3070404 , 0.98801979, 0.44567367, 0.19211148,
       0.9953147 , 0.26849204, 0.94067574, 0.18614681, 0.44718872,
       0.14234202, 0.25236196, 0.76068446, 0.08646759, 0.62321276])

In [144]:
np.tile(np.array([[1,0,0], [0,1,0], [0,0,1]])[..., np.newaxis], N)

array([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]])

In [110]:
np.arange(3*21).reshape(3,21)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41],
       [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
        58, 59, 60, 61, 62]])

In [142]:
np.arange(3*21).reshape(21,3)[..., np.newaxis].repeat(3, axis=2)

array([[[ 0,  0,  0],
        [ 1,  1,  1],
        [ 2,  2,  2]],

       [[ 3,  3,  3],
        [ 4,  4,  4],
        [ 5,  5,  5]],

       [[ 6,  6,  6],
        [ 7,  7,  7],
        [ 8,  8,  8]],

       [[ 9,  9,  9],
        [10, 10, 10],
        [11, 11, 11]],

       [[12, 12, 12],
        [13, 13, 13],
        [14, 14, 14]],

       [[15, 15, 15],
        [16, 16, 16],
        [17, 17, 17]],

       [[18, 18, 18],
        [19, 19, 19],
        [20, 20, 20]],

       [[21, 21, 21],
        [22, 22, 22],
        [23, 23, 23]],

       [[24, 24, 24],
        [25, 25, 25],
        [26, 26, 26]],

       [[27, 27, 27],
        [28, 28, 28],
        [29, 29, 29]],

       [[30, 30, 30],
        [31, 31, 31],
        [32, 32, 32]],

       [[33, 33, 33],
        [34, 34, 34],
        [35, 35, 35]],

       [[36, 36, 36],
        [37, 37, 37],
        [38, 38, 38]],

       [[39, 39, 39],
        [40, 40, 40],
        [41, 41, 41]],

       [[42, 42, 42],
        [43, 43, 43],
    

In [112]:
C = rgb_points
N = C.size//3

In [120]:
N

10

In [132]:
onecol = np.tile([[1,0,0],[0,1,0],[0,0,1]], [N,1])
onecol

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [119]:
#C_d = C.repeat(3, axis=0) + 

In [123]:
help(np.repeat)

Help on function repeat in module numpy:

repeat(a, repeats, axis=None)
    Repeat elements of an array.
    
    Parameters
    ----------
    a : array_like
        Input array.
    repeats : int or array of ints
        The number of repetitions for each element.  `repeats` is broadcasted
        to fit the shape of the given axis.
    axis : int, optional
        The axis along which to repeat values.  By default, use the
        flattened input array, and return a flat output array.
    
    Returns
    -------
    repeated_array : ndarray
        Output array which has the same shape as `a`, except along
        the given axis.
    
    See Also
    --------
    tile : Tile an array.
    unique : Find the unique elements of an array.
    
    Examples
    --------
    >>> np.repeat(3, 4)
    array([3, 3, 3, 3])
    >>> x = np.array([[1,2],[3,4]])
    >>> np.repeat(x, 2)
    array([1, 1, 2, 2, 3, 3, 4, 4])
    >>> np.repeat(x, 3, axis=1)
    array([[1, 1, 1, 2, 2, 2],
           [

In [263]:
sRGBColor(*(100,100,100), is_upscaled=False).get_value_tuple()

(100.0, 100.0, 100.0)

In [248]:
help(sRGBColor)

Help on class sRGBColor in module colormath.color_objects:

class sRGBColor(BaseRGBColor)
 |  sRGBColor(rgb_r, rgb_g, rgb_b, is_upscaled=False)
 |  
 |  Represents an sRGB color.
 |  
 |  .. note:: If you pass in upscaled values, we automatically scale them
 |      down to 0.0-1.0. If you need the old upscaled values, you can
 |      retrieve them with :py:meth:`get_upscaled_value_tuple`.
 |  
 |  :ivar float rgb_r: R coordinate
 |  :ivar float rgb_g: G coordinate
 |  :ivar float rgb_b: B coordinate
 |  :ivar bool is_upscaled: If True, RGB values are between 1-255. If False,
 |      0.0-1.0.
 |  
 |  Method resolution order:
 |      sRGBColor
 |      BaseRGBColor
 |      ColorBase
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  conversion_matrices = {'rgb_to_xyz': array([[0.412424 , 0.357579 , 0.1...
 |  
 |  native_illuminant = 'd65'
 |  
 |  rgb_gamma = 2.2
 |  
 |  ----------------------------------------------------------------------
 |  Methods in